In [1]:
%load_ext nbextension
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr

from sklearn.naive_bayes import GaussianNB
import tensorflow as tf

from gswp.constants import STORE, PROBSEVERE as PS, GMGSI


print(
    [
        tf.config.experimental.get_device_details(gpu)
        for gpu in tf.config.list_physical_devices("GPU")
    ]
)

2022-10-18 22:59:04.627017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 22:59:04.864587: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-18 22:59:05.513061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-18 22:59:05.513182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

[{'compute_capability': (7, 5), 'device_name': 'NVIDIA GeForce RTX 2080 SUPER'}]


2022-10-18 22:59:06.491159: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-18 22:59:06.526567: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-18 22:59:06.526631: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-18 22:59:06.526916: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
features = (
    PS.load()  # .to_dataframe()
    # .rename(columns={"MINX": "minx", "MINY": "miny", "MAXX": "maxx", "MAXY": "maxy"})
)
features

<xarray.Dataset>
Dimensions:            (time: 17139030)
Coordinates:
  * time               (time) datetime64[ns] 2022-03-01T00:00:29 ... 2022-09-...
Data variables: (12/28)
    CAPE_M10M30        (time) uint32 dask.array<chunksize=(5192,), meta=np.ndarray>
    EBSHEAR            (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    FLASH_DENSITY      (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    FLASH_RATE         (time) uint32 dask.array<chunksize=(5192,), meta=np.ndarray>
    ID                 (time) uint32 dask.array<chunksize=(5192,), meta=np.ndarray>
    LJA                (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    ...                 ...
    maxx               (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    maxy               (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    minx               (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    miny               (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    x                  (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>
    y                  (time) float32 dask.array<chunksize=(5192,), meta=np.ndarray>

In [3]:
gmgsi = GMGSI.load()
gmgsi

<xarray.Dataset>
Dimensions:    (lat: 3000, lon: 4999, time: 5053)
Coordinates:
  * lat        (lat) float32 -72.74 -72.72 -72.69 -72.67 ... 72.67 72.69 72.72
  * lon        (lon) float32 -179.9 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * time       (time) datetime64[ns] 2022-03-04 ... 2022-09-30T23:00:00
Data variables:
    GMGSI_LW   (time, lat, lon) float32 dask.array<chunksize=(1, 3000, 4999), meta=np.ndarray>
    GMGSI_WV   (time, lat, lon) float32 dask.array<chunksize=(1, 3000, 4999), meta=np.ndarray>
    GMGSI_VIS  (time, lat, lon) float32 dask.array<chunksize=(1, 3000, 4999), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.4
    NCO:                  netCDF Operators version 4.7.5 (Homepage = http://n...
    Satellite Sensor:     DERIVED DATA
    Source:               McIDAS Area File
    history:              Fri Sep 30 00:38:28 2022: ncks -d xc,0,4998 tempo.n...
    instrument_name:      GLOBCOMPLIR
    time_coverage_start:  2022-09-30T00:00:00

In [4]:
from typing import Generic, NewType, TypeVar
from gswp.constants import MRMS_BOUNDS
from gswp.api import minimum_diffrence

W, E, S, N = MRMS_BOUNDS
_1D = NewType("1d", tuple)
_2D = NewType("2d", tuple)


def load_and_filter() -> pd.DataFrame:
    gmgsi = GMGSI.load()
    lat, lon = gmgsi["lat"].to_numpy(), gmgsi["lon"].to_numpy()
    lat = lat[(lat > S) & (lat < N)]
    lon = lon[(lon > W) & (lon < E)]
    return gmgsi.sel({"lat": lat, "lon": lon})


def min_diff(
    target: np.ndarray[_1D, np.floating], values: np.ndarray[_1D, np.floating]
) -> np.ndarray[_1D, np.unsignedinteger]:
    diff = abs(target[:, np.newaxis] - values)
    index = np.argmin(diff, axis=0)
    return index


gmgsi = load_and_filter()
lat, lon = (np.unique(gmgsi[crd]) for crd in ("lat", "lon"))
gmgsi

<xarray.Dataset>
Dimensions:    (lat: 635, lon: 972, time: 5053)
Coordinates:
  * lat        (lat) float32 20.03 20.1 20.16 20.23 ... 54.86 54.9 54.94 54.98
  * lon        (lon) float32 -130.0 -129.9 -129.8 ... -60.18 -60.11 -60.03
  * time       (time) datetime64[ns] 2022-03-04 ... 2022-09-30T23:00:00
Data variables:
    GMGSI_LW   (time, lat, lon) float32 dask.array<chunksize=(1, 635, 972), meta=np.ndarray>
    GMGSI_WV   (time, lat, lon) float32 dask.array<chunksize=(1, 635, 972), meta=np.ndarray>
    GMGSI_VIS  (time, lat, lon) float32 dask.array<chunksize=(1, 635, 972), meta=np.ndarray>
Attributes:
    Conventions:          CF-1.4
    NCO:                  netCDF Operators version 4.7.5 (Homepage = http://n...
    Satellite Sensor:     DERIVED DATA
    Source:               McIDAS Area File
    history:              Fri Sep 30 00:38:28 2022: ncks -d xc,0,4998 tempo.n...
    instrument_name:      GLOBCOMPLIR
    time_coverage_start:  2022-09-30T00:00:00

In [5]:
min_lon = minimum_diffrence(lon, features.minx.to_numpy(), engine="dask")
max_lon = minimum_diffrence(lon, features.maxx.to_numpy(), engine="dask")
min_lat = minimum_diffrence(lat, features.miny.to_numpy(), engine="dask")
max_lat = minimum_diffrence(lat, features.maxy.to_numpy(), engine="dask")

MemoryError: Unable to allocate 62.1 GiB for an array with shape (972, 17139030) and data type float32

In [ ]:
# NOTE:  NOT CORRECT!!! the times are not aligned
data = gmgsi.set_index(["lat", "lon"], append=True).unstack("lon")


def extract(arr):
    return [
        arr[..., x1:x2, y1:y2]
        for x1, x2, y1, y2 in np.c_[min_lat, max_lat, min_lon, max_lon]
    ]


arr = np.array(
    [
        data["GMGSI_LW"].to_numpy(),
        data["GMGSI_SW"].to_numpy(),
        data["GMGSI_WV"].to_numpy(),
    ]
)


features["observations"] = extract(arr)


features

In [ ]:
np.array(
    [
        data["GMGSI_LW"].unstack("lon").to_numpy(),
        data["GMGSI_SW"].unstack("lon").to_numpy(),
        data["GMGSI_WV"].unstack("lon").to_numpy(),
    ]
).shape

In [ ]:
import matplotlib.pyplot as plt

components = [
    "CAPE_M10M30",
    "EBSHEAR",
    "FLASH_DENSITY",
    "FLASH_RATE",
    "LJA",
    "MAXLLAZ",
    "MEANWIND_1-3kmAGL",
    "MESH",
    "MLCAPE",
    "MLCIN",
    "MUCAPE",
    "P98LLAZ",
    "P98MLAZ",
    "PS",
    "PWAT",
    # "SIZE",
    "SRH01KM",
    "VIL_DENSITY",
    "WETBULB_0C_HGT",
]

for label, s in (
    features[features["PS"] > 60]
    .sort_values("SIZE", ascending=False)
    .head(20)
    .iterrows()
):
    extent = s[["minx", "maxx", "miny", "maxy"]].to_list()
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(18, 5))
    #
    storm_id = s["ID"]
    ax1.set_title(f"{storm_id=} longwave")
    ax1.imshow(s["longwave"], extent=extent)
    #
    ax2.set_title(f"{storm_id=} shortwave")
    ax2.imshow(s["shortwave"], extent=extent)
    #
    ax3.set_title(f"{storm_id=} shortwave")
    ax3.imshow(s["watervapor"], extent=extent)
    #
    s[components].plot.bar(ax=ax4)